In [1]:
import pandas as pd
import numpy as np
import re
import os
import string
import io
import gensim
from gensim import models
import nltk
import warnings
import math
import csv
from textblob import TextBlob
from gensim import corpora
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from gensim.models import LdaModel
from sklearn.model_selection import train_test_split
from nltk.classify import SklearnClassifier
#from sklearn.feature_extraction.text import TdifTransformer, CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from string import punctuation
%matplotlib inline

In [2]:
df = pd.read_csv(r'1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
df.columns = ['polarity','ID','date','query','user','text']
df.head()

,polarity,ID,date,query,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [3]:
import collections
from collections import Counter
collections.Counter(df['polarity'])


Counter({0: 799999, 4: 800000})

In [4]:
to_drop = ['ID','query','user','date']
df.drop(to_drop, inplace=True, axis=1)
df

,polarity,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1599994,4,Just woke up. Having no school is the best fee...
1599995,4,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,Happy 38th Birthday to my boo of alll time!!! ...


In [5]:
def remove_mentionUrls(text):
    tweet_out = re.sub(r'@[A-Za-z0-9]+','',text)
    re.sub('https?://[A-Za-z0-9./]+','',tweet_out)
    return tweet_out
                       
def remove_nonalphanumeric(text):
    text_out = "".join([char for char in text if char not in string.punctuation])
    return text_out

In [ ]:
df['no ment url'] = df['text'].apply(lambda x: remove_mentionUrls(x))
df['nopunc'] = df['no ment url'].apply(lambda x: remove_nonalphanumeric(x))
df

In [ ]:
#def tokanization(text):
 #   tokens = re.split('\W+',text)
  #  return tokens

#df['tweet'] = df['nopunc'].apply(lambda x:tokanization(x.lower()))
#df

In [ ]:
to_drop = ['no ment url','text']
df.drop(to_drop, inplace=True, axis=1)
df

In [ ]:
negative = df[df.polarity < 4]
train_neg = negative.iloc[:400000] 
train_negative = train_neg.sample(15000)
train_negative

In [ ]:
test_neg = negative.iloc[400000:]
test_negative = test_neg.sample(3000)
test_negative

In [ ]:
positive = df[df.polarity >= 4]
train_pos = positive.iloc[:400000] 
train_positive = train_pos.sample(15000)
train_positive

In [ ]:
test_pos = positive.iloc[400000:]
test_positive = test_pos.sample(3000)
test_positive

In [ ]:
training = pd.concat((train_negative, train_positive), axis=0)
training

In [ ]:
testset = pd.concat((test_negative, test_positive), axis=0)
testset

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn import metrics


In [ ]:
feature1 = []
for index, row in training.iterrows():
    feature1.append((row["polarity"], row["nopunc"]))
feature1

In [ ]:
feature2 = []
for index, row in testset.iterrows():
    feature2.append((row["polarity"], row["nopunc"]))
feature2

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

# Generate counts from text using a vectorizer.  There are other vectorizers available, and lots of options you can set.
# This performs our step of computing word counts.
vectorizer = CountVectorizer(stop_words='english')
train_features = vectorizer.fit_transform([r[0] for r in feature1])
test_features = vectorizer.transform([r[0] for r in feature2])

nb = MultinomialNB()
nb.fit(train_features, [int(r[1]) for r in feature1])

# Now we can use the model to predict classifications for our test features.
predictions = nb.predict(feature2)